# Serving a Resnet TF Estimator Model

**Scenario:** In Tensorflow 1.3, a higher level API called Estimators was introduced and has since been a popular API of choice within the Tensorflow community. Suppose that an ML researcher has trained a Resnet model on the Imagenet dataset using Tensorflow's Estimator API, located at https://github.com/tensorflow/models/tree/v1.4.0/official/resnet. (Note that we used v1.4.0. You always want to use a stable tag for a model version to deploy as the researcher can continue to modify the model and architecture at the head of master.) Our task is to deploy this model into Tensorflow Serving. You have access to their python code as well as a saved state (checkpoint) that points to their favorite trained result.

The first step is to create a servable version of the model that will be used for Tensorflow Serving, which runs very efficiently in C++, and is platform independent (can run on different OSes, as well as hardware with different types of accelerators such as GPUs).

# Preamble

Import the required libraries.

In [0]:
import os
import tensorflow as tf

# Download model checkpoint

The next step is to load the researcher's saved checkpoint into our estimator. We will download it from
http://download.tensorflow.org/models/official/resnet50_2017_11_30.tar.gz using the commands below.


In [0]:
# Define a constant indicating the number of layers in our loaded model. We're loading a resnet-50 model.
RESNET_SIZE = 50  

# Model and serving directories
MODEL_DIR="resnet_model_checkpoints"
SERVING_DIR="resnet_servable"

In [0]:
import urllib.request

urllib.request.urlretrieve("http://download.tensorflow.org/models/official/resnet50_2017_11_30.tar.gz ", "resnet.tar.gz")

In [0]:
#unzip the file into a directory called resnet
from subprocess import call
call(["mkdir", MODEL_DIR])
call(["tar", "-zxvf", "resnet.tar.gz", "-C", MODEL_DIR])

In [0]:
# Make sure you see model checkpoint files in this directory
os.listdir(MODEL_DIR)

# Import the Model Architecture
 
In order to reconstruct the Resnet neural network used to train the Imagenet model, we need to load the architecture pieces. During the setup step, we checked out https://github.com/tensorflow/models/tree/v1.4.0/official/resnet. We can now load functions and constants from resnet_model.py into the notebook.

**Exercise:** We also need some constants from [imagenet_main.py](https://github.com/tensorflow/models/blob/v1.4.0/official/resnet/imagenet_main.py), but we cannot run this file as it is a main class that trains ResNet. Open [imagenet_main.py](https://github.com/tensorflow/models/blob/v1.4.0/official/resnet/imagenet_main.py) and copy over a few constants that are important, such as the image size, channels, and number of classes, in the cell below with the TODO.

In [0]:
%run ../models/official/resnet/resnet_model.py

In [0]:
# TODO: Copy constants from imagenet_main.py.
#
# Hint: You do not need to copy all constants, such as those pertaining only to training and validation.
# What constants are useful for prediction?
#
# Hint 2: What constants are referenced in the code cells below for prediction? You may come back to edit this cell.



# Preprocess JPEG images into 3D Tensors

In order to reduce network overhead, we've provided a client (resnet_client.py) that will encode images into an array of jpegs (encoded as byte strings) to send to the server. These jpegs are all appropriately resized to 224x224x3, and do not need resizing on the server side to enter into the ResNet model.

**Exercise:** Create a helper function that decodes a jpeg image, and normalizes pixel values to be between -0.5 and 0.5.


In [0]:
def preprocess_image(encoded_image, height=_DEFAULT_IMAGE_SIZE, width=_DEFAULT_IMAGE_SIZE):
  """Preprocesses the image by subtracting out the mean from all channels.
  Args:
    image: A jpeg-formatted byte stream represented as a string.
  Returns:
    A 3d tensor of image pixels normalized to be between -0.5 and 0.5, resized to height x width x 3.
    The normalization is an approximation of the preprocess_for_train and preprocess_for_eval functions in
    https://github.com/tensorflow/models/blob/v1.4.0/official/resnet/vgg_preprocessing.py.
  """
  image = ???  # TODO: Use a tf function to decode the jpeg into a 3d tensor.
  image = tf.to_float(image) / 255.0 - 0.5  # Normalize values to be between -0.5 and 0.5.
  return image

# Build the Servable from the Estimator API

Within the Tensorflow Estimator API, a model function defines graph elements used in training, evaluation, and prediction. Depending on the mode (TRAIN, EVAL, PREDICT) used, the model function will return an [EstimatorSpec](https://www.tensorflow.org/versions/r1.3/extend/estimators#constructing_the_model_fn) object tell the Estimator to run different graph elements. For instance:

* TRAIN mode would usually involve calling an optimizer that is hooked to a loss function (e.g. cross-entropy), which depends on the logits for each class, which depends on lower layers of the network, etc.
* EVAL mode would not call the optimizer, but would call the loss function and/or some other evaluation metric (e.g. accuracy). These evaluation metrics will likely depend on labels as well as the logits of the network output, which depends on lower layers of the network, etc.
  * Additionally, researchers will often use monitors and hooks during training and evaluation to check on the progress of the model. Usually, these components are used to return summaries about different layers of the network, such as model coefficients, etc., which can be visualized using [Tensorboard](https://www.tensorflow.org/programmers_guide/summaries_and_tensorboard).
* PREDICT mode does NOT require an optimizer as there is no training step. Also, input data is unlabeled, so no loss functions or evaluation metrics are used. Instead, predictions simply try to provide clients/users with information of interest, such as the most likely label for an image, the probability of the image being of a particular class, etc.

**Exercise:** The model server is primarily used to PREDICT. Therefore, many graph elements in the training code located in the imagenet_main.py [resnet_model_fn()](https://github.com/tensorflow/models/blob/v1.4.0/official/resnet/imagenet_main.py#L162) are no longer relevant. We are going to modify the function in several ways:

1. Remove/shortcut many graph elements unrelate to prediction.

2. Change the way input features are read to be compliant with our [client](https://github.com/google-aai/tf-serving-k8s-tutorial/blob/master/client/resnet_client.py). In particular, training data is prepackaged as a [TF Dataset](https://www.tensorflow.org/programmers_guide/datasets) with information such as labels, text, encoding, bounding boxes, etc. Our client's behavior is very simple: we send a message with a single field 'images' containing a batch (array) of jpeg-encoded images as strings.

3. Modify the `prediction` value to return the top K likely classes and their probabilities! This may be more useful than returning just the top class and probability, as it helps the user determine whether the model is confused about a particular image's class!

**Useful References:**
* [tf.map_fn](https://www.tensorflow.org/api_docs/python/tf/map_fn)
* [tf.DType](https://www.tensorflow.org/api_docs/python/tf/DType)
* [ResNet Model Code](https://github.com/tensorflow/models/blob/v1.4.0/official/resnet/resnet_model.py)
* [tf.nn.top_k](https://www.tensorflow.org/api_docs/python/tf/nn/top_k)
* [tf.estimator.EstimatorSpec](https://www.tensorflow.org/api_docs/python/tf/estimator/EstimatorSpec)

In [0]:
TOP_K = 5

# This is a modified version of the original code with exercises involved. SERVING CODE indicates changes made to
# enable serving a batch of images provided by the client
def resnet_model_fn(features, labels, mode, params):
  """Our model_fn for ResNet to be used with our Estimator."""
  tf.summary.image('images', features, max_outputs=6)  # TODO: What is tf.summary used for, and what kind of format is it expecting?

  ## SERVING CODE  
  # TODO: the features received from our client is simply a dictionary {'images' : tensor_of_jpeg_byte_strings}.
  # Use tf.map_fn and the preprocess_image() helper function above to preprocess the tensor_of_jpeg_byte_strings
  # into a list of single-precision floating point 3D tensors.
  images = features['images']
  processed_images = ???
  processed_images = tf.stack(processed_images)  # Convert list of 3D tensors to a 4D tensor
  processed_images = tf.reshape(tensor=processed_images,  # Reshaping informs Tensorflow of the final dimensions of the 4D tensor
                                shape=[-1, _DEFAULT_IMAGE_SIZE, _DEFAULT_IMAGE_SIZE, 3])
  ## END SERVING CODE

  # TODO: You are building a servable model that is optimized toward your particular cluster type. 
  # For CNNs, it has been shown that placing your color channels ('channels_first') before your pixel dimensions 
  # in the image tensor significantly improves performance over 'channels_last'. 
  # 
  # Feel free to try out both data formats, i.e. 'channels_first' and 'channels_last',
  # to compare performances during serving.
  #
  # HOWEVER, validating the servable (i.e. `resnet_servable_validation.ipynb`) REQUIRES 'channels_last'
  # due to limitations in the tf.contrib.predict package. If you want to validate your servable,
  # we suggest you start by creating a servable with data format 'channels_last' for validation,
  # then recreate a servable with 'channels_first' as this should also work without issues.
  network = imagenet_resnet_v2(???)
  
  # TODO: Change inputs to processed_images instead of features
  logits = network(
      inputs=features, is_training=(mode == tf.estimator.ModeKeys.TRAIN))

  # TODO: Instead of the top 1 result for classes and probabilities, return the top k. The dictionary should
  # contain the same keys, i.e.:
  # 'classes': 1D-tensor of top k classes
  # 'probabilities': 1D-tensor of top k probabilities.
  #
  # HINT: Use tf.nn.top_k.
  predictions = {
      'classes': tf.argmax(logits, axis=1),
      'probabilities': tf.nn.softmax(logits, name='softmax_tensor')
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    ## SERVING CODE
    # We refactored the original prediction EstimatorSpec into a function to be created in the next exercise.
    return create_servable_estimator_spec(predictions, mode)  # This line is purposely modified from the original. See cells below.
    ## END SERVING CODE

    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # TODO: You already returned the EstimatorSpec for predictions!
  # Shortcut every graph element below here by returning a minimal EstimatorSpec.

  # Calculate loss, which includes softmax cross entropy and L2 regularization.
  cross_entropy = tf.losses.softmax_cross_entropy(
      logits=logits, onehot_labels=labels)

  # Create a tensor named cross_entropy for logging purposes.
  tf.identity(cross_entropy, name='cross_entropy')
  tf.summary.scalar('cross_entropy', cross_entropy)

  # Add weight decay to the loss. We exclude the batch norm variables because
  # doing so leads to a small improvement in accuracy.
  loss = cross_entropy + _WEIGHT_DECAY * tf.add_n(
      [tf.nn.l2_loss(v) for v in tf.trainable_variables()
       if 'batch_normalization' not in v.name])

  if mode == tf.estimator.ModeKeys.TRAIN:
    # Scale the learning rate linearly with the batch size. When the batch size
    # is 256, the learning rate should be 0.1.
    initial_learning_rate = 0.1 * params['batch_size'] / 256
    batches_per_epoch = _NUM_IMAGES['train'] / params['batch_size']
    global_step = tf.train.get_or_create_global_step()

    # Multiply the learning rate by 0.1 at 30, 60, 80, and 90 epochs.
    boundaries = [
        int(batches_per_epoch * epoch) for epoch in [30, 60, 80, 90]]
    values = [
        initial_learning_rate * decay for decay in [1, 0.1, 0.01, 1e-3, 1e-4]]
    learning_rate = tf.train.piecewise_constant(
        tf.cast(global_step, tf.int32), boundaries, values)

    # Create a tensor named learning_rate for logging purposes.
    tf.identity(learning_rate, name='learning_rate')
    tf.summary.scalar('learning_rate', learning_rate)

    optimizer = tf.train.MomentumOptimizer(
        learning_rate=learning_rate,
        momentum=_MOMENTUM)

    # Batch norm requires update_ops to be added as a train_op dependency.
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
      train_op = optimizer.minimize(loss, global_step)
  else:
    train_op = None

  accuracy = tf.metrics.accuracy(
      tf.argmax(labels, axis=1), predictions['classes'])
  metrics = {'accuracy': accuracy}

  # Create a tensor named train_accuracy for logging purposes.
  tf.identity(accuracy[1], name='train_accuracy')
  tf.summary.scalar('train_accuracy', accuracy[1])

  return tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=predictions,
      loss=loss,
      train_op=train_op,
      eval_metric_ops=metrics)

# Servable Model API Definition

The EstimatorSpec contains a field `export_outputs`, which defines the dictionary of fields that the servable model will return to a client upon receiving a request. To export the predictions dictionary above using Tf serving, you will need to assign the export_outputs parameter in EstimatorSpec.

**Exercise:** Add a dictionary with a string key which will be the request.model_spec.signature_name that
your client will call in `client/resnet_client.py`. (See client/resnet_client.py)
Add a value for the key that is tf.estimator.export.PredictOutput(outputs=predictions)

In [0]:
def create_servable_estimator_spec(predictions, mode):
  return tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=predictions,  # Note: This is not be used in serving, but must be provided for the Estimator API.
      # TODO: assign an appropriate dictionary to the export_outputs parameter here.
  )

In [0]:
# Load this model into our estimator
estimator = tf.estimator.Estimator(
  model_fn=resnet_model_fn,  # Call our generate_model_fn to create model function
  model_dir=MODEL_DIR,  # Where to look for model checkpoints
)

# Serving input receiver function

Tensorflow uses [placeholders](https://www.tensorflow.org/api_docs/python/tf/placeholder) as entrypoints for ingesting data into its graphs. For example, if you create a placeholder with dimensions [100, 50], you can pass in a numpy array of dimension (100, 50), and run the graph with the placeholder substituted by a 2d tensor with the numpy array values. However, this requires several steps, such as creating a [Tensorflow Session, passing in a feed dictionary, and running the session](https://www.tensorflow.org/api_docs/python/tf/Session).

The same principle applies for Tensorflow serving, except that the placeholder holds values sent to the server from the client. Tensorflow Estimator API simplifies the process above by providing "serving input receiver functions" as will be seen in the below exercise.

**Exercise**: Recall that your client is sending a message of the format:

```
{'images': array_of_jpeg_encoded_images_as_strings}
```

Add the following argument to [build_raw_serving_input_receiver_fn()](https://www.tensorflow.org/api_docs/python/tf/estimator/export/build_parsing_serving_input_receiver_fn) below: a dictionary with a single key 'images', and value [tf.placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) that can read in an arbitrary length array of strings.

**Hint:** You need to define the `shape` parameter in tf.placeholder. `None` inside an array indicates that the length can vary along that dimension.



In [0]:
def serving_input_receiver_fn():
  return tf.estimator.export.build_raw_serving_input_receiver_fn(???)()  ## TODO: Add dictionary

In [0]:
estimator.export_savedmodel(export_dir_base=SERVING_DIR,
                            serving_input_receiver_fn=serving_input_receiver_fn)